# A/B Testing for ShoeFly.com

Our favorite online shoe store, ShoeFly.com is performing an A/B Test. They have two different versions of an ad, which they have placed in emails, as well as in banner ads on Facebook, Twitter, and Google. They want to know how the two ads are performing on each of the different platforms on each day of the week. Help them analyze the data using aggregate measures.

Examine the first few rows of ad_clicks

In [2]:
import pandas as pd

ad_clicks = pd.read_csv('ad_clicks.csv')
print(ad_clicks.head())

                                user_id utm_source           day  \
0  008b7c6c-7272-471e-b90e-930d548bd8d7     google  6 - Saturday   
1  009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook    7 - Sunday   
2  00f5d532-ed58-4570-b6d2-768df5f41aed    twitter   2 - Tuesday   
3  011adc64-0f44-4fd9-a0bb-f1506d2ad439     google   2 - Tuesday   
4  012137e6-7ae7-4649-af68-205b4702169c   facebook    7 - Sunday   

  ad_click_timestamp experimental_group  
0               7:18                  A  
1                NaN                  B  
2                NaN                  A  
3                NaN                  B  
4                NaN                  B  


Your manager wants to know which ad platform is getting you the most views. How many views (i.e., rows of the table) came from each utm_source?

In [3]:
print(ad_clicks.groupby('utm_source').user_id.count().reset_index())

  utm_source  user_id
0      email      255
1   facebook      504
2     google      680
3    twitter      215


If the column ad_click_timestamp is not null, then someone actually clicked on the ad that was displayed. Create a new column called is_click, which is True if ad_click_timestamp is not null and False otherwise.

In [4]:
ad_clicks['is_click']=ad_clicks.ad_click_timestamp.isnull()
print(ad_clicks)

                                   user_id utm_source            day  \
0     008b7c6c-7272-471e-b90e-930d548bd8d7     google   6 - Saturday   
1     009abb94-5e14-4b6c-bb1c-4f4df7aa7557   facebook     7 - Sunday   
2     00f5d532-ed58-4570-b6d2-768df5f41aed    twitter    2 - Tuesday   
3     011adc64-0f44-4fd9-a0bb-f1506d2ad439     google    2 - Tuesday   
4     012137e6-7ae7-4649-af68-205b4702169c   facebook     7 - Sunday   
...                                    ...        ...            ...   
1649  fe8b5236-78f6-4192-9da6-a76bba67cfe6    twitter     7 - Sunday   
1650  fed3db6d-8c92-40e3-a4fb-1fb9d7337eb1   facebook     5 - Friday   
1651  ff3a22ff-521c-478c-87ca-7dc7b8f34372    twitter  3 - Wednesday   
1652  ff3af0d6-b092-4c4d-9f2e-2bdd8f7c0732     google     1 - Monday   
1653  ffdfe7ec-0c74-4623-8d90-d95d80f1ba34   facebook   6 - Saturday   

     ad_click_timestamp experimental_group  is_click  
0                  7:18                  A     False  
1                   NaN  

We want to know the percent of people who clicked on ads from each utm_source. Start by grouping by utm_source and is_click and counting the number of user_id‘s in each of those groups. Save your answer to the variable clicks_by_source

In [5]:
clicks_by_source=ad_clicks.groupby(['utm_source','is_click'])['user_id'].count().reset_index()
print(clicks_by_source)

  utm_source  is_click  user_id
0      email     False       80
1      email      True      175
2   facebook     False      180
3   facebook      True      324
4     google     False      239
5     google      True      441
6    twitter     False       66
7    twitter      True      149


Now let’s pivot the data so that the columns are is_click (either True or False), the index is utm_source, and the values are user_id. Save your results to the variable clicks_pivot

In [6]:
clicks_pivot=clicks_by_source.pivot(
  columns='is_click',
  index= 'utm_source',
  values='user_id'
).reset_index()
print(clicks_pivot)

is_click utm_source  False  True
0             email     80   175
1          facebook    180   324
2            google    239   441
3           twitter     66   149


Create a new column in clicks_pivot called percent_clicked which is equal to the percent of users who clicked on the ad from each utm_source. Was there a difference in click rates for each source?

In [7]:
clicks_pivot['percent_clicked']=clicks_pivot[True]*100/(clicks_pivot[True] + clicks_pivot[False])
print(clicks_pivot)

is_click utm_source  False  True  percent_clicked
0             email     80   175        68.627451
1          facebook    180   324        64.285714
2            google    239   441        64.852941
3           twitter     66   149        69.302326


The column experimental_group tells us whether the user was shown Ad A or Ad B. Were approximately the same number of people shown both adds?

In [9]:
Add_A_or_B=ad_clicks.groupby('experimental_group').user_id.count().reset_index()
print(Add_A_or_B)

  experimental_group  user_id
0                  A      827
1                  B      827


From the above,it is clear that approximately the same number of users were shown ad A or ad B

Using the column is_click that we defined earlier, check to see if a greater percentage of users clicked on Ad A or Ad B

In [10]:
percent_ad_click=ad_clicks.groupby(['experimental_group','is_click'])['user_id'].count().reset_index()

In [11]:
print(percent_ad_click.pivot(
  columns='is_click',
  index='experimental_group',
  values='user_id'
).reset_index())

is_click experimental_group  False  True
0                         A    310   517
1                         B    255   572


Yes a greater percentage of users clicked on the ads

The Product Manager for the A/B test thinks that the clicks might have changed by day of the week. Start by creating two DataFrames: a_clicks and b_clicks, which contain only the results for A group and B group, respectively.

In [12]:
a_clicks = ad_clicks[
   ad_clicks.experimental_group
   == 'A']
b_clicks = ad_clicks[
   ad_clicks.experimental_group
   == 'B']

For each group (a_clicks and b_clicks), calculate the percent of users who clicked on the ad by day

In [14]:
clicked_A=a_clicks.groupby(['is_click','day']).user_id.count().reset_index().pivot(
  columns='is_click',
  index='day',
  values='user_id').reset_index()

clicked_B=b_clicks.groupby(['is_click','day']).user_id.count().reset_index().pivot(
  columns='is_click',
  index='day',
  values='user_id').reset_index()
print(clicked_A)

print(clicked_B)

is_click            day  False  True
0            1 - Monday     43    70
1           2 - Tuesday     43    76
2         3 - Wednesday     38    86
3          4 - Thursday     47    69
4            5 - Friday     51    77
5          6 - Saturday     45    73
6            7 - Sunday     43    66
is_click            day  False  True
0            1 - Monday     32    81
1           2 - Tuesday     45    74
2         3 - Wednesday     35    89
3          4 - Thursday     29    87
4            5 - Friday     38    90
5          6 - Saturday     42    76
6            7 - Sunday     34    75
